Environment Set-up

In [2]:
import pandas as pd
import pyreadstat
from sas7bdat import SAS7BDAT
import numpy as np
from tabulate import tabulate

Importing Data

In [3]:
acs_raw = pd.read_csv("Data/ACS_5YR/2018_2022/psam_p06.csv")

In [ ]:
with SAS7BDAT('Data/CHIS Dummy/Adult 2022/dummy_adult.sas7bdat') as file:
    chis_raw = file.to_data_frame()

In [ ]:
acs_raw['SERIALNO'].head()

Data Manipulation Toolbox - DataToolBox

In [ ]:
class DataToolBox:
    def __init__(self, data):
        """
        Initialize the DataToolBox with a dataset.

        :param data: A pandas DataFrame that contains the data to be analyzed and manipulated.
        """
        self.data = data

    def return_data(self):
        """
        Return the current state of the data stored in the toolbox.

        :return: The current pandas DataFrame stored within the tool.
        """
        return self.data

    def data_desc(self):
        """
        Print a description of the current dataset including the number of observations (rows) and variables (columns).
        """
        temp = self.data.shape
        print("---------Current Data State----------")
        print(temp[0], "obs;", temp[1], "vars")
        print("")

    def data_exclude(self, condition: str):
        """
        Exclude observations from the data based on a given condition and updates the dataset.

        :param condition: A string representing the condition to be used for filtering the data.
                          Observations meeting this condition will be excluded.
        """
        temp = self.data.query(condition)
        temp_new_obs = temp.shape[0]
        temp_old_obs = self.data.shape[0]
        temp_diff_obs = temp_old_obs - temp_new_obs

        print("---------Obs Filter-----------------")
        print("applying condition: ", condition)
        print(temp_diff_obs, "/", temp_old_obs, "cases were removed")
        print("new obs #: ", temp_new_obs)
        print("")

        self.data = temp

    def freq_1way(self, col_name):
        """
        Print the frequency count and percentage distribution of a single column, sorted by index,
        and display it in a formatted table with borders. Rows with a count of zero are not shown.

        :param col_name: The name of the column for which the frequency distribution is to be calculated.
        """
        # Get counts and sort by index
        counts = self.data[col_name].value_counts().sort_index()
        # Calculate percentages
        percentages = (counts / counts.sum()) * 100
        # Combine counts and percentages into a single DataFrame for better display
        frequency_df = pd.DataFrame({
            'Counts': counts,
            'Percentage': percentages
        })

        # Filter out rows with zero counts and reset the index
        frequency_df = frequency_df[frequency_df['Counts'] > 0].reset_index()

        # Print results using tabulate for better formatting
        print("---------Frequency Distribution for", col_name, "----------")
        print(
            tabulate(frequency_df,
                     headers='keys',
                     tablefmt='grid',
                     showindex=False))
        print("")

    def freq_2way(self, col_name_1, col_name_2, exclude_equal=False):
        """
        Print a two-way frequency table (crosstab) between two columns in a formatted table with borders,
        excluding rows where the count is zero, and optionally excluding rows where values in A are equal to B.
        Rows with a count of zero are not indexed.

        :param col_name_1: The name of the first column.
        :param col_name_2: The name of the second column.
        :param exclude_equal: If True, exclude rows where the values of col_name_1 are equal to col_name_2.
        """
        # Generate crosstab data
        crosstab_result = pd.crosstab(self.data[col_name_1],
                                      self.data[col_name_2],
                                      rownames=[col_name_1],
                                      colnames=[col_name_2])

        # Reset index to make crosstab a regular DataFrame
        crosstab_result = crosstab_result.reset_index()

        # Melt the DataFrame to get a long format
        melted_crosstab = crosstab_result.melt(id_vars=[col_name_1],
                                               value_name='COUNT')

        # Filter out rows where COUNT is zero and optionally where values in column A are equal to values in column B
        if exclude_equal:
            melted_crosstab = melted_crosstab[
                (melted_crosstab['COUNT'] > 0)
                & (melted_crosstab[col_name_1] != melted_crosstab[col_name_2])]
            title = f"Two-way Frequency Table for {col_name_1} and {col_name_2} (Unequal Values Only)"
        else:
            melted_crosstab = melted_crosstab[melted_crosstab['COUNT'] > 0]
            title = f"Two-way Frequency Table for {col_name_1} and {col_name_2}"

        # Reset index to clean up the table view
        melted_crosstab = melted_crosstab.reset_index(drop=True)

        # Sort the results based on column A
        melted_crosstab = melted_crosstab.sort_values(
            by=[col_name_1, col_name_2])

        # Print results using tabulate for better formatting
        print("---------", title, "----------")
        print(
            tabulate(melted_crosstab,
                     headers='keys',
                     tablefmt='grid',
                     showindex=False))
        print("")

    def freq_multiway(self, columns, exclude_zeros=True, exclude_equal=False):
        """
        Print a multi-way frequency table (crosstab) between given columns in a formatted table with borders,
        excluding rows where the count is zero, and optionally excluding rows where all values in specified columns are equal.

        :param columns: List of column names to include in the frequency table.
        :param exclude_zeros: If True, exclude rows where the count is zero.
        :param exclude_equal: If True, exclude rows where all specified column values are equal.
        """
        # Generate crosstab data
        crosstab_result = pd.crosstab(
            index=[self.data[col] for col in columns[:-1]],
            columns=self.data[columns[-1]],
            rownames=columns[:-1],
            colnames=[columns[-1]])

        # Reset index to make crosstab a regular DataFrame and flatten MultiIndex
        crosstab_result.reset_index(inplace=True)
        melted_crosstab = pd.melt(crosstab_result,
                                  id_vars=columns[:-1],
                                  value_name='COUNT')

        # Apply filters
        if exclude_zeros:
            melted_crosstab = melted_crosstab[melted_crosstab['COUNT'] > 0]

        if exclude_equal and len(columns) > 1:
            # Check if all elements in each row are equal
            equal_filter = melted_crosstab.apply(lambda row: len(
                set(row[columns[:-1]].tolist() + [row[columns[-1]]])) == 1,
                                                 axis=1)
            melted_crosstab = melted_crosstab[~equal_filter]

        # Sort the results
        melted_crosstab.sort_values(by=columns, inplace=True)

        # Print results using tabulate for better formatting
        title = f"Multi-way Frequency Table for {' ,'.join(columns)}"
        if exclude_equal:
            title += " (Non-equal Values Only)"
        print("---------", title, "----------")
        print(
            tabulate(melted_crosstab,
                     headers='keys',
                     tablefmt='grid',
                     showindex=False))
        print("")

    def data_construct(self, col_name, conditions_str, choices, default=-1):
        """
        Construct a new column in the data based on multiple conditions,
        where choices can be either fixed values or column references.
        Stop checking further conditions once a true condition is met for a row.

        :param col_name: Name of the new column to be added.
        :param conditions_str: A list of conditions (as strings) that determine the value to be assigned.
        :param choices: A list of values or column names to be assigned based on the conditions.
        :param default: The default value or column name to be assigned if none of the conditions are met. Default is -1.
        """
        temp_df = self.data.copy()

        # Initialize the new column with NaNs which will be replaced by the default at the end
        temp_df[col_name] = np.nan

        # Loop through each condition and choice, assign only if the column is still NaN
        for condition, choice in zip(conditions_str, choices):
            condition_series = temp_df.eval(condition)
            if isinstance(choice, str) and choice in temp_df.columns:
                # Apply choice from another column
                temp_df.loc[condition_series & temp_df[col_name].isna(),
                            col_name] = temp_df.loc[condition_series, choice]
            else:
                # Apply fixed choice
                temp_df.loc[condition_series & temp_df[col_name].isna(),
                            col_name] = choice

        # Fill remaining NaNs with the default value
        if isinstance(default, str) and default in temp_df.columns:
            temp_df[col_name].fillna(temp_df[default], inplace=True)
        else:
            temp_df[col_name].fillna(default, inplace=True)

        self.data = temp_df
    def copy_column(self, source_col, target_col):
        """
        Copies the values from one column to another, preserving the original column.

        :param source_col: The name of the source column whose values are to be copied.
        :param target_col: The name of the target column to which the values will be copied.
        """
        if source_col in self.data.columns:
            self.data[target_col] = self.data[source_col]
            print(f"Values from '{source_col}' were successfully copied to '{target_col}'.")
        else:
            print(f"Error: The column '{source_col}' does not exist in the DataFrame.")

In [ ]:
acs_working = acs_raw.copy()

conditions = [
    acs_raw['SERIALNO'].str[4:6] == 'GQ',  # Condition for 'GQ'
    acs_raw['SERIALNO'].str[4:6] == 'HU'   # Condition for 'HU'
]

choices = [1, 0]

acs_working['INGRPQ'] = np.select(conditions, choices, default = -1)

ACS Processing

In [ ]:
acs = DataToolBox(acs_working)
acs.data_desc()
acs.data_exclude('INGRPQ == 0')

CHIS Processing

In [ ]:
chis = DataToolBox(chis_raw)
chis.data_desc()

In [ ]:
acs.data_construct("sc_sex", ['SEX == 1', "SEX == 2"], [1,2])
chis.data_construct('sc_sex', ["SRSEX == 1", "SRSEX == 2"], [1,2])

acs.freq_2way('SEX', "sc_sex")
chis.freq_2way('SRSEX', "sc_sex")

In [ ]:
acs.data_construct("sc_age_cont", ['AGEP <=99'], ['AGEP'])
chis.data_construct('sc_age_cont', ['SRAGE < 99', 'SRAGE >= 99'], ['SRAGE', 99])

acs.freq_2way('AGEP','sc_age_cont', exclude_equal=True)
chis.freq_2way('SRAGE', 'sc_age_cont', exclude_equal=True)

In [ ]:
acs.data_construct('sc_age_cat', [
    'AGEP < 18', 'AGEP >= 18 & AGEP < 25', 'AGEP >= 25 & AGEP < 35',
    'AGEP >= 35 & AGEP < 45', 'AGEP >= 45 & AGEP < 55',
    'AGEP >= 55 & AGEP < 64', 'AGEP >= 65'
], list(range(7)))
chis.data_construct('sc_age_cat', [
    'SRAGE < 18', 'SRAGE >= 18 & SRAGE < 25', 'SRAGE >= 25 & SRAGE < 35',
    'SRAGE >= 35 & SRAGE < 45', 'SRAGE >= 45 & SRAGE < 55',
    'SRAGE >= 55 & SRAGE < 64', 'SRAGE >= 65'
], list(range(7)))
acs.freq_2way('AGEP', 'sc_age_cat')
chis.freq_2way('SRAGE', 'sc_age_cat')

In [ ]:
acs.data_construct('sc_hisp', ['HISP == 1', 'HISP != 1'], [1, 2])
acs.freq_2way('HISP', 'sc_hisp')

chis.data_construct('sc_hisp', ['SRH == 1', 'SRH != 1'], [1, 2])
chis.freq_2way('SRH', 'sc_hisp')

In [ ]:
acs.freq_1way('RAC1P')

In [ ]:
acs.data_construct('sc_race_ethi', [
    'HISP == 1', 'RAC1P == 1', 'RAC1P == 2', 'RAC1P == 3 | RAC1P == 4 |RAC1P == 5',
    'RAC1P == 6', 'RAC1P == 7', 'RAC1P == 8 | RAC1P == 9'
], [1, 2, 3, 4, 5, 6, 7])

chis.copy_column('OMBSRREO','sc_race_ethi')

acs.freq_multiway(['sc_hisp', 'RAC1P','sc_race_ethi'])
chis.freq_2way('OMBSRREO', 'sc_race_ethi')

In [ ]:
acs.data_construct('sc_cit', ['CIT == 1|CIT == 2|CIT==3', 'CIT ==4', 'CIT==5'],
                   [1, 2, 3])
chis.copy_column('CITIZEN2', "sc_cit")

acs.freq_2way('CIT', 'sc_cit')
chis.freq_2way('CITIZEN2', 'sc_cit')

acs.freq_1way('sc_cit')
chis.freq_1way('sc_cit')

In [ ]:
acs.freq_1way('SCHL')

In [ ]:
acs.data_construct('sc_edu', ['SCHL >= 4 & SCHL <= 11', 
                              'SCHL >= 12 & SCHL <= 14',
                               'SCHL >= 15 & SCHL <= 17',
                                'SCHL >= 18 & SCHL <= 19', 
                                  'SCHL == 20'], 
                   
                   [1, 2, 3, 4, 6])
acs.freq_2way("SCHL", 'sc_edu')

In [209]:
chis.freq_2way('SREDUC', 'AHEDUC')

--------- Two-way Frequency Table for SREDUC and AHEDUC ----------
+----------+----------+---------+
|   SREDUC |   AHEDUC |   COUNT |
+==========+==========+=========+
|        1 |        1 |     372 |
+----------+----------+---------+
|        1 |        2 |     493 |
+----------+----------+---------+
|        1 |       91 |      75 |
+----------+----------+---------+
|        2 |        3 |    2489 |
+----------+----------+---------+
|        3 |        4 |    2928 |
+----------+----------+---------+
|        3 |        5 |    1286 |
+----------+----------+---------+
|        3 |        6 |    1514 |
+----------+----------+---------+
|        4 |        7 |    6399 |
+----------+----------+---------+
|        4 |        8 |     602 |
+----------+----------+---------+
|        4 |        9 |    3703 |
+----------+----------+---------+
|        4 |       10 |    1602 |
+----------+----------+---------+

